# Head Detection and Localization with Haar Cascades

A script for detecting and localizing heads in video clip frames using Haar Cascades.

The Haar Cascade used in this script is downloaded from [https://github.com/opencv/opencv/tree/master/data/haarcascades](https://github.com/opencv/opencv/tree/master/data/haarcascades).
Note that it's important that the *raw* version of the Haar Cascade be downloaded.

In [1]:
import os
import re
import pickle

import cv2

In [2]:
def load_rgb_img(img_path):
    """
    Takes in img_path, the file path to an image.
    Reads the image at that path, and returns the RGB image.
    """
    img_raw = cv2.imread(img_path)
    
    # OpenCV imread reads in images in BGR color space, so must convert from BGR to RGB
    img_rgb = cv2.cvtColor(img_raw, cv2.COLOR_BGR2RGB)
    
    return img_rgb

In [3]:
def detectFeatureUsingHaarCascade(img_path, haarcascade_filename):
    """
    Detects a feature in an image, using a Haar Cascade.
    
    Parameters:
    - img_path: the filepath to an image.
    - haarcascade_filename: a filename that encodes a Haar Cascade for a certain feature, ends with a ".xml" extension
                            (e.g. "haarcascade_frontalface_default.xml").

    Writes a new version of the input image with a bounding box for each instance of the feature detected using the Haar Cascade.
    Returns the number of instances of the feature detected by the Haar Cascade.
    """
    _, clip_frames_folder_name, img_name = img_path.split("/")
    
    img = load_rgb_img(img_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) # convert to grayscale to use Haar Cascade
    f_cascade = cv2.CascadeClassifier()
    f_cascade.load(cv2.samples.findFile(haarcascade_filename)) # the Haar Cascade file must be located in the same directory as this Jupyter Notebook
    detected = f_cascade.detectMultiScale(gray, 1.3, 5) # TODO: tweak parameters?
    img_head_localization = img.copy() # make a copy of original image so original image is retained while we draw bounding boxes on the copy
    # For each detected instance of the feature...
    for (x,y,w,h) in detected:
        # Draw a bounding box around the instance of the feature
        img_head_localization = cv2.rectangle(img_head_localization,(x,y),(x+w,y+h),(0,194,255),2) # TODO: change rectangle border color?
    
    # If a directory for frames with head bounding boxes detected using Haar does not currently exist, make that directory
    head_localization_dir_path = "head_localization_haar"
    if not os.path.isdir(head_localization_dir_path):
        ! mkdir {head_localization_dir_path}
    
    # If a directory for current clip's head bounding box frames does not currently exist, make that directory
    clip_head_localization_dir_path = "head_localization_haar/" + clip_frames_folder_name
    if not os.path.isdir(clip_head_localization_dir_path):
        ! mkdir {clip_head_localization_dir_path}

    # OpenCV imwrite expects BGR image, so convert from RGB to BGR color space before using OpenCV's imwrite function to save it
    cv2.imwrite('head_localization_haar/' + clip_frames_folder_name + "/" + img_name, cv2.cvtColor(img_head_localization, cv2.COLOR_RGB2BGR))
    return len(detected)

In [4]:
# Get list of all files and directories in frames directory
frames_dir_path = "frames"
clips_list = os.listdir(frames_dir_path)

# For each clip's frames folder...
for clip_folder in clips_list:
    if clip_folder != ".DS_Store":
        # Get list of all files and directories in the clip's frames directory
        clip_frames_dir_path = frames_dir_path + "/" + clip_folder
        frames_list = os.listdir(clip_frames_dir_path)

        # Dictionary that will store head detection results using Haar Cascades
        # Keys are frame numbers, and values are whether or not the frame contains a head (0 if frame doesn't contain a head; 1 if it does).
        head_label_dict = {}

        # For each frame...
        for frame_filename in frames_list:
            # We only want to detect heads in our frames, which are jpg files
            if frame_filename.endswith(".jpg"):
                frame_num = int(frame_filename.split("_")[-1].replace(".jpg", ""))
                # Detect if there's a head in the frame, using frontal face Haar Cascade
                # TODO: TRY USING MULTIPLE HAAR CASCADES; if any succeed, say there's a head and break out of loop
                if detectFeatureUsingHaarCascade(clip_frames_dir_path + "/" + frame_filename, "haarcascade_frontalface_default.xml") > 0:
                    head_label_dict[frame_num] = 1
                else:
                    head_label_dict[frame_num] = 0

        # If a directory for head label dictionaries detected using Haar Cascades does not currently exist, make that directory
        head_dict_haar_dir_path = "head_label_dicts_haar"
        if not os.path.isdir(head_dict_haar_dir_path):
            ! mkdir {head_dict_haar_dir_path}

        # Save (serialize) pickle of the Haar Cascade head detection result dictionary
        pickled_dict_filepath = head_dict_haar_dir_path + "/" + clip_folder.replace("_frames", "") + "_frame_to_head_dict" + ".pkl"
        with open(pickled_dict_filepath, "wb") as f:
            pickle.dump(head_label_dict, f)

        # Test loading (deserializing) pickled data to make sure pickled file saved correctly
        with open(pickled_dict_filepath, "rb") as f:
             print(pickle.load(f))


{1684: 0, 1848: 0, 1690: 0, 2399: 1, 1860: 0, 1874: 0, 1647: 0, 559: 0, 2428: 1, 1121: 0, 1135: 0, 1653: 0, 2372: 1, 565: 0, 203: 0, 2414: 1, 2400: 1, 217: 0, 1109: 0, 571: 0, 2366: 1, 2831: 0, 1486: 0, 940: 0, 798: 0, 1492: 0, 954: 0, 2825: 0, 968: 0, 2819: 0, 1323: 0, 1445: 0, 983: 0, 1451: 0, 997: 0, 2158: 0, 1337: 0, 29: 0, 2616: 0, 1: 0, 767: 0, 1479: 0, 2170: 0, 2164: 0, 773: 0, 15: 0, 2602: 0, 1294: 0, 834: 0, 820: 0, 1280: 0, 2789: 0, 808: 0, 149: 0, 1257: 0, 2038: 0, 1531: 0, 1525: 0, 1243: 0, 2762: 0, 175: 0, 613: 0, 2004: 0, 2010: 0, 1519: 0, 607: 0, 161: 0, 2776: 0, 1928: 0, 388: 0, 1096: 0, 1082: 0, 1914: 0, 1900: 0, 1733: 0, 1055: 0, 1041: 0, 2548: 0, 439: 0, 1727: 0, 2206: 0, 411: 0, 1069: 0, 377: 0, 2560: 0, 2574: 0, 363: 0, 405: 0, 2212: 0, 362: 0, 2575: 0, 2213: 0, 404: 0, 410: 0, 2207: 0, 2561: 0, 376: 0, 1068: 0, 2549: 0, 1040: 0, 1726: 0, 438: 0, 1732: 0, 1054: 0, 1901: 0, 1915: 0, 1083: 0, 1929: 0, 1097: 0, 389: 0, 606: 0, 1518: 0, 2011: 0, 2777: 0, 160: 0, 174: 0

{918: 0, 2869: 1, 1390: 0, 2841: 1, 2699: 0, 930: 0, 924: 0, 2855: 1, 1384: 0, 71: 0, 2666: 0, 2100: 1, 1409: 0, 717: 0, 703: 0, 2114: 1, 2672: 0, 65: 0, 2882: 1, 1353: 0, 1435: 0, 3222: 1, 3236: 1, 2128: 1, 1421: 0, 59: 0, 1347: 0, 2896: 0, 1810: 1, 1804: 1, 1192: 0, 298: 0, 1186: 0, 1838: 1, 515: 0, 2302: 0, 2464: 1, 273: 0, 1179: 0, 267: 0, 2470: 1, 2316: 0, 3008: 1, 501: 0, 3020: 1, 529: 0, 1637: 0, 1151: 0, 2458: 1, 1145: 0, 1623: 0, 3034: 1, 1964: 1, 1970: 1, 1958: 0, 1780: 0, 2289: 0, 3197: 1, 3183: 0, 1794: 1, 461: 0, 3168: 0, 2276: 0, 2510: 0, 307: 0, 1019: 0, 313: 0, 2504: 0, 2262: 1, 475: 1, 3154: 0, 1743: 1, 1025: 0, 2538: 0, 1031: 0, 1757: 0, 449: 0, 3140: 1, 2909: 0, 878: 0, 2935: 0, 3395: 1, 844: 0, 1582: 0, 850: 0, 1596: 0, 688: 0, 3381: 1, 2921: 0, 105: 0, 2712: 0, 2074: 1, 663: 0, 677: 0, 1569: 0, 2060: 0, 3418: 1, 2706: 0, 111: 0, 3430: 1, 1227: 0, 139: 0, 887: 0, 1541: 0, 2048: 0, 3356: 1, 3342: 0, 893: 0, 1555: 0, 1233: 0, 3424: 1, 1554: 0, 892: 0, 3343: 1, 3425: 0

{3967: 0, 1816: 1, 1802: 1, 3973: 0, 1194: 1, 3783: 1, 4952: 0, 4946: 0, 2489: 0, 3797: 0, 1180: 0, 275: 0, 4013: 0, 2462: 0, 2304: 0, 4775: 0, 513: 0, 507: 0, 1619: 0, 2310: 0, 4761: 0, 3768: 0, 4007: 0, 2476: 0, 261: 0, 3740: 0, 3998: 0, 1157: 1, 249: 0, 1631: 0, 2338: 1, 4991: 0, 4749: 0, 3026: 0, 4985: 0, 3032: 0, 1625: 0, 1143: 1, 3754: 0, 936: 0, 4588: 0, 2847: 0, 3581: 0, 1396: 0, 88: 0, 1382: 1, 2853: 0, 3595: 0, 922: 0, 711: 1, 3218: 1, 4577: 0, 2106: 0, 2660: 0, 4211: 0, 77: 0, 1369: 1, 63: 0, 2674: 0, 4205: 0, 4563: 0, 2112: 0, 705: 1, 3224: 0, 1433: 0, 1355: 1, 5133: 0, 2884: 0, 3542: 0, 2648: 0, 5127: 0, 4239: 0, 2890: 0, 3556: 0, 1341: 1, 1427: 0, 739: 0, 3230: 0, 842: 0, 1584: 0, 3393: 1, 5084: 0, 2933: 0, 5090: 0, 2927: 0, 3387: 0, 2099: 0, 856: 1, 1590: 0, 665: 0, 2072: 1, 4403: 1, 4365: 0, 2714: 0, 103: 0, 1209: 0, 117: 0, 4371: 0, 2700: 0, 2066: 0, 4417: 0, 3378: 0, 671: 0, 3350: 1, 659: 1, 881: 1, 1547: 0, 1221: 0, 3436: 0, 4359: 0, 5047: 0, 2728: 0, 3422: 1, 5053: 

{1684: 0, 3093: 0, 3939: 0, 2399: 0, 3087: 0, 1848: 0, 1690: 0, 1860: 1, 3911: 0, 3905: 0, 1874: 0, 3050: 0, 1647: 0, 559: 0, 1121: 0, 2428: 0, 3736: 0, 4059: 0, 3722: 0, 1135: 0, 1653: 0, 3044: 0, 565: 0, 2372: 0, 2414: 0, 4065: 0, 203: 0, 217: 0, 1109: 0, 2400: 0, 4071: 0, 3078: 0, 2366: 0, 571: 0, 2831: 0, 3291: 0, 1486: 0, 940: 0, 798: 1, 1492: 0, 954: 0, 3285: 0, 2825: 0, 968: 0, 2819: 0, 3534: 0, 1323: 0, 1445: 0, 983: 0, 3252: 0, 2158: 0, 3246: 0, 1451: 0, 997: 0, 1337: 0, 29: 0, 3520: 0, 1: 1, 4267: 1, 3508: 0, 2616: 0, 2170: 0, 767: 1, 1479: 0, 773: 1, 2164: 0, 4273: 1, 2602: 0, 15: 1, 1294: 0, 3483: 1, 2945: 0, 834: 0, 820: 0, 3497: 1, 2951: 0, 2789: 0, 1280: 0, 2979: 0, 808: 0, 3440: 0, 2986: 0, 149: 0, 1257: 0, 1531: 0, 3326: 0, 2038: 1, 3332: 0, 1525: 0, 1243: 0, 3454: 1, 2992: 0, 175: 0, 2762: 1, 2004: 0, 613: 1, 1519: 0, 607: 1, 2010: 0, 2776: 0, 3468: 1, 161: 0, 1928: 0, 3859: 0, 3681: 0, 388: 0, 1096: 0, 1082: 0, 3695: 0, 1914: 0, 3865: 0, 3871: 0, 1900: 0, 3124: 0, 17

{2709: 0, 1200: 0, 1566: 0, 678: 1, 1572: 0, 1214: 0, 122: 0, 2735: 0, 2053: 0, 644: 1, 888: 0, 650: 1, 2047: 0, 2721: 0, 136: 0, 1228: 0, 2912: 0, 863: 0, 877: 0, 2906: 0, 687: 0, 1599: 0, 2090: 0, 2084: 0, 693: 0, 3173: 0, 1764: 0, 1002: 0, 1016: 0, 308: 0, 1770: 0, 2279: 0, 3167: 1, 446: 0, 1980: 0, 1758: 0, 2251: 0, 2537: 0, 320: 0, 334: 0, 2523: 0, 2245: 0, 452: 0, 1994: 0, 2292: 0, 485: 0, 1943: 1, 491: 0, 1957: 1, 2286: 0, 3007: 0, 2319: 0, 1610: 0, 268: 1, 1176: 0, 1162: 0, 1604: 0, 3013: 0, 532: 0, 2325: 0, 2443: 0, 254: 0, 240: 0, 2457: 0, 2331: 0, 1638: 1, 526: 0, 1837: 1, 1189: 0, 297: 1, 2480: 0, 2494: 0, 283: 1, 1823: 1, 1374: 0, 1412: 0, 718: 0, 1406: 0, 1360: 1, 2669: 0, 1348: 0, 56: 0, 2899: 0, 2641: 0, 2127: 1, 730: 0, 724: 0, 2133: 1, 2655: 0, 42: 0, 2866: 0, 917: 0, 903: 0, 2872: 0, 2682: 0, 95: 0, 81: 0, 2696: 0, 2697: 0, 80: 0, 94: 0, 2683: 0, 902: 0, 2873: 0, 2867: 0, 916: 0, 2132: 1, 725: 0, 43: 0, 2654: 0, 2640: 1, 2898: 0, 57: 0, 1349: 0, 731: 0, 2126: 1, 1407

...

{974: 1, 960: 1, 790: 0, 948: 0, 784: 0, 9: 0, 1317: 1, 1303: 1, 35: 0, 753: 1, 747: 1, 21: 0, 592: 1, 586: 1, 1115: 0, 1101: 0, 579: 1, 551: 0, 1129: 0, 237: 0, 223: 0, 545: 0, 380: 1, 394: 1, 419: 1, 1061: 1, 1075: 1, 425: 1, 343: 1, 357: 1, 1049: 1, 431: 1, 800: 0, 814: 0, 182: 1, 828: 1, 196: 1, 1288: 0, 1263: 1, 1277: 1, 169: 1, 141: 0, 627: 1, 633: 1, 155: 0, 632: 1, 154: 0, 140: 0, 626: 1, 168: 1, 1276: 1, 1262: 1, 829: 1, 1289: 1, 197: 1, 183: 1, 815: 0, 801: 0, 1048: 1, 356: 1, 430: 1, 424: 1, 342: 1, 1074: 1, 418: 1, 1060: 1, 395: 1, 381: 1, 222: 0, 544: 0, 550: 0, 236: 0, 1128: 0, 1100: 0, 578: 1, 1114: 0, 587: 1, 593: 1, 746: 1, 20: 0, 34: 0, 752: 0, 1302: 1, 1316: 1, 8: 0, 785: 0, 949: 0, 791: 0, 961: 1, 975: 1, 963: 1, 977: 0, 787: 0, 793: 0, 1300: 1, 778: 0, 1314: 1, 22: 0, 744: 1, 988: 1, 750: 0, 36: 0, 1328: 1, 585: 1, 591: 1, 1102: 0, 1116: 0, 208: 1, 546: 0, 220: 0, 234: 0, 552: 1, 1089: 0, 397: 1, 383: 1, 368: 1, 1076: 1, 1062: 1, 432: 0, 354: 1, 340: 1, 426: 1, 817

{973: 1, 967: 1, 2180: 1, 1489: 1, 797: 0, 783: 1, 2194: 0, 1310: 1, 768: 1, 1476: 0, 1462: 1, 1304: 1, 32: 1, 754: 1, 2143: 1, 2157: 1, 740: 1, 998: 1, 1338: 0, 26: 1, 595: 1, 1853: 1, 581: 1, 1847: 0, 1674: 1, 1112: 0, 218: 1, 1106: 0, 1660: 1, 1648: 1, 556: 1, 1890: 1, 230: 1, 224: 1, 542: 0, 1884: 1, 1927: 1, 387: 1, 1099: 1, 393: 1, 1933: 1, 1700: 1, 2209: 0, 1066: 1, 378: 1, 1072: 1, 1714: 1, 2235: 0, 422: 1, 344: 0, 350: 1, 436: 1, 1728: 1, 2221: 0, 807: 0, 813: 0, 185: 1, 191: 1, 1264: 0, 1502: 0, 1516: 0, 608: 1, 1270: 0, 146: 1, 1258: 0, 620: 1, 2037: 0, 2023: 1, 634: 0, 152: 1, 635: 0, 2022: 1, 153: 1, 1259: 0, 147: 1, 2036: 0, 621: 0, 609: 1, 1517: 0, 1271: 0, 1265: 0, 1503: 0, 190: 1, 184: 1, 812: 0, 806: 0, 351: 1, 2220: 0, 1729: 1, 437: 1, 423: 1, 2234: 0, 345: 0, 1073: 1, 1715: 1, 2208: 0, 1701: 1, 379: 1, 1067: 1, 392: 1, 1932: 1, 1926: 1, 1098: 1, 386: 1, 225: 1, 1885: 1, 543: 0, 1891: 1, 557: 1, 1649: 1, 231: 1, 1107: 1, 219: 1, 1661: 1, 1675: 1, 1113: 0, 1846: 1, 58

{2922: 0, 3382: 0, 853: 0, 1595: 0, 847: 0, 1581: 0, 3396: 0, 2088: 0, 2936: 0, 3427: 0, 2739: 0, 1230: 1, 648: 0, 890: 0, 1556: 0, 3341: 0, 3355: 0, 884: 0, 1542: 0, 1224: 0, 3433: 0, 112: 0, 2705: 0, 2063: 0, 674: 0, 660: 0, 2077: 0, 3369: 0, 2711: 0, 1218: 0, 106: 0, 489: 0, 1797: 0, 3180: 0, 3194: 0, 1783: 0, 1973: 0, 3802: 0, 3816: 0, 1967: 0, 3143: 1, 1754: 0, 1032: 0, 3625: 0, 3631: 0, 338: 0, 1026: 0, 1740: 0, 1998: 0, 3157: 1, 2249: 1, 1768: 0, 476: 0, 2261: 0, 2507: 0, 3619: 0, 310: 0, 304: 0, 2513: 0, 2275: 0, 462: 0, 3792: 1, 1185: 0, 1191: 0, 2498: 0, 3786: 0, 1807: 0, 1813: 0, 2329: 0, 3037: 0, 1620: 0, 1146: 1, 258: 0, 3751: 0, 3745: 0, 1152: 0, 1634: 0, 3023: 0, 502: 0, 2315: 0, 2473: 0, 264: 0, 270: 0, 3779: 0, 2467: 0, 2301: 0, 516: 0, 1608: 0, 1387: 0, 99: 0, 2856: 0, 3590: 0, 927: 0, 933: 0, 2842: 0, 3584: 1, 1393: 0, 2895: 0, 3553: 0, 1344: 0, 1422: 0, 3235: 0, 3221: 0, 1436: 0, 728: 0, 1350: 0, 2659: 0, 2881: 0, 3547: 0, 66: 0, 1378: 0, 2671: 0, 3209: 0, 2117: 0, 

{2741: 1, 2999: 1, 156: 0, 1248: 0, 630: 0, 3339: 0, 2027: 1, 2033: 1, 624: 0, 142: 0, 2755: 0, 1274: 1, 3463: 0, 3305: 0, 1512: 0, 1506: 0, 618: 0, 3311: 0, 2769: 1, 3477: 0, 1260: 0, 195: 0, 2782: 0, 3488: 0, 2796: 0, 181: 0, 2966: 0, 817: 0, 803: 0, 2972: 0, 2225: 1, 432: 0, 354: 0, 2543: 0, 2557: 0, 340: 0, 426: 0, 1738: 0, 2231: 1, 1710: 0, 2219: 1, 3107: 1, 1076: 1, 368: 0, 1062: 1, 3113: 1, 1704: 0, 1937: 0, 2580: 1, 397: 0, 1089: 1, 383: 0, 2594: 1, 1923: 0, 2351: 0, 1658: 0, 1880: 0, 546: 0, 220: 0, 2437: 0, 2423: 0, 234: 0, 1894: 0, 552: 0, 2345: 0, 1664: 0, 3073: 1, 1102: 1, 208: 0, 1116: 1, 3067: 1, 2379: 0, 1670: 0, 1843: 0, 585: 0, 2392: 0, 2386: 0, 3098: 1, 1857: 0, 591: 0, 2635: 0, 22: 0, 744: 0, 2153: 0, 2147: 0, 3259: 0, 750: 0, 988: 1, 1328: 0, 36: 0, 2621: 0, 1300: 0, 3517: 0, 2609: 0, 3271: 0, 778: 0, 1466: 0, 1472: 0, 3265: 0, 3503: 0, 1314: 0, 2190: 0, 1499: 0, 787: 0, 793: 0, 2184: 0, 2812: 0, 963: 1, 977: 1, 2806: 0, 976: 1, 2807: 0, 2813: 0, 962: 1, 2185: 0, 7

{1775: 1, 1013: 0, 1007: 0, 319: 0, 1761: 1, 2268: 0, 457: 0, 1991: 1, 1749: 1, 2240: 0, 2526: 0, 331: 0, 325: 0, 2532: 0, 2254: 0, 443: 0, 1985: 1, 2283: 0, 494: 1, 1952: 0, 480: 1, 1946: 0, 2297: 0, 2718: 0, 1211: 0, 1577: 0, 669: 0, 1563: 0, 1205: 0, 133: 0, 2724: 0, 2042: 0, 655: 0, 899: 0, 641: 0, 2056: 0, 2730: 0, 127: 0, 1239: 0, 2903: 0, 872: 0, 866: 0, 2917: 0, 696: 0, 1588: 0, 2081: 0, 2095: 0, 682: 0, 1365: 0, 1403: 0, 709: 0, 1417: 1, 1371: 0, 2678: 0, 1359: 0, 47: 0, 2888: 0, 2650: 0, 2136: 0, 721: 0, 735: 0, 2122: 1, 2644: 0, 53: 0, 2877: 0, 906: 0, 912: 0, 2863: 0, 2693: 0, 84: 0, 90: 0, 2687: 0, 2308: 1, 1601: 0, 279: 0, 1167: 0, 1173: 0, 1615: 1, 523: 1, 2334: 0, 2452: 0, 245: 0, 251: 0, 2446: 1, 2320: 1, 1629: 1, 537: 1, 1826: 0, 1198: 0, 286: 0, 2491: 0, 2485: 0, 292: 0, 1832: 0, 293: 0, 2484: 1, 1833: 0, 1827: 0, 2490: 0, 287: 0, 1199: 0, 2447: 1, 250: 0, 536: 1, 1628: 1, 2321: 1, 2335: 0, 522: 1, 244: 0, 2453: 1, 1172: 0, 1614: 1, 1600: 0, 2309: 1, 1166: 0, 278: 0,